## 💻ETL para Credits Dataset 🎥
En el siguiente notebook se presentan las transformaciones requeridas para obtener un dataset limpio y confiable el cual va ser cargado a un modelo de recomendación.


### Importación librerias 📜

Las librerias importadas son las necesarias para realizar con exito el proceso ETL, algunas librerias se eligen debido a las caracteristicas del dataset, por ejemplo el modulo **literal_eval** de la libreria **ast** se usa debido a que los datos de algunas columnas vienen como string pero en realidad representan una estructura de datos más compleja como un dict o un list.

In [143]:
# Importación de librerías requeridas.
import numpy as np 
import pandas as pd
from ast import literal_eval # Importamos este modulo para convertir String a Dict y a List.
import datetime as dt #Esta libreria nos ayuda a trabajar con los campos de tipo fecha.

Ahora leemos los datos desde un archivo CSV usando el metodo read_csv que provee pandas. En este paso hay que tener cuidado con la ruta donde se encuentra almacenado el archivos csv, para este caso accedemos de forma local.

In [144]:
# Usando la función read_csv de Pandas leemos y almacenamos credits_dataset como un dataframe con el nombre credits_data.
credits_data = pd.read_csv('../datasets/credits.csv', low_memory=False)

Previsualizamos las 5 primeras filas del dataframe así cómo las 5 últimas.

In [145]:
# Con la ayuda del metodo head echamos un vistazo preeliminar al dataframe creado.
credits_data.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [146]:
# Con la ayuda del metodo tail echamos un vistazo preeliminar al dataframe creado.
credits_data.tail()

,cast,crew,id
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506
45475,[],"[{'credit_id': '593e676c92514105b702e68e', 'de...",461257


Con el metodo info() echamos un vistazo al dataset completo.

In [147]:
credits_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


Se procede a inspeccionar la estructura de anidación de los datos contenidos en las columnas **'crew'** y **'cast'**.

In [148]:
# Imprimimos el tipo un campo para cada columna que dice el diccionario de datos que están anidados para visualizar su estructura, luego imprimimos el typo de dato para evidenciar cómo viene formateado.
print(credits_data['cast'][2])
print(credits_data['crew'][2])


print(type(credits_data['cast'][2]))
print(type(credits_data['crew'][2]))


[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2, 'id': 6837, 'name': 'Walter Matthau', 'order': 0, 'profile_path': '/xJVkvprOnzP5Zdh5y63y8HHniDZ.jpg'}, {'cast_id': 3, 'character': 'John Gustafson', 'credit_id': '52fe466a9251416c75077a91', 'gender': 2, 'id': 3151, 'name': 'Jack Lemmon', 'order': 1, 'profile_path': '/chZmNRYMtqkiDlatprGDH4BzGqG.jpg'}, {'cast_id': 4, 'character': 'Ariel Gustafson', 'credit_id': '52fe466a9251416c75077a95', 'gender': 1, 'id': 13567, 'name': 'Ann-Margret', 'order': 2, 'profile_path': '/jx5lTaJ5VXZHYB52gaOTAZ9STZk.jpg'}, {'cast_id': 5, 'character': 'Maria Sophia Coletta Ragetti', 'credit_id': '52fe466a9251416c75077a99', 'gender': 1, 'id': 16757, 'name': 'Sophia Loren', 'order': 3, 'profile_path': '/emKLhbji1c7BjcA2DdbWf0EP9zH.jpg'}, {'cast_id': 6, 'character': 'Melanie Gustafson', 'credit_id': '52fe466a9251416c75077a9d', 'gender': 1, 'id': 589, 'name': 'Daryl Hannah', 'order': 4, 'profile_path': '/4LLmp6AQdlj6u

### Duplicados 🧱
Antes de proceder se realiza primero la elimincaión de duplicados.

In [149]:
credits_data = credits_data.drop_duplicates(subset=['id'])

### Desanidando 🐦
Se procede a desanidar los campos de las columnas 'cast', 'crew'. El metodo usado es el resultado de varios pruebas realizadas con anterioridad para determinar cual es el mejor, inicialmente durante la exploración preeliminar se evidencia que los datos anidados vienen en tipo **str** y con el modulo **literal_eval** de la libreria **ast** se logra convertir a la forma ya sea **dict** o **list**, posteriormente se procede a desanidar y almacenar en dataframes independientes usando el modulo **Series** de Pandas, este metodo se elige ya que se debe cumplir con rapidez con la tarea asignada y no vamos a reinventar la rueda, simplemente usamos lo que ya existe.

In [150]:
'''Aplicamos una función Lambda a cada columna que viene anidada, dentro de la función lambda se aplica a cada campo la funcion
literal_eval para que trasnforme el string en su estrucutura adecuada ya sea dict o list, en el caso de encontrar un valor nulo se reemplaza con np.nanksk'''

credits_data['cast'] = credits_data['cast'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)
credits_data['crew'] = credits_data['crew'].apply(lambda x: literal_eval(x) if pd.notna(x) else np.nan)

### Cast 🤵
En la exploración inicial y en el diccionario de los datos nos muestran que la columna 'cast' viene como una lista de diccionarios ya que una pelicula puede ser clasificada dentro de varios generos, para proceder a desanidarlo lo primero que se realiza es guardar dentro de una variable independiente las columnas generadas al aplicar el metodo explode el  cual nos ayuda a sacar cada elemento de la lista de diccionarios y los almacena en un dataframe, despues aplicamos a ese diccionario el metodo Series.

In [151]:
#Con el metodo explode() se transforma el dato anidado
cast_df = credits_data['cast'].explode().apply(pd.Series)
cast_df.head()

,cast_id,character,credit_id,gender,id,name,order,profile_path,0
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,NaN
0,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,NaN
0,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,NaN
0,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,NaN
0,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,NaN


Vamos a mantener sólo la columna 'name' y avamos a aagrupar por indice y almacenamos como una cadena separada pot coma con el metodo join. Luego renombramos la columna 'name' por 'actors'.

In [152]:
cast_df = cast_df[['name']]
# Agrupar las filas por el índice original y unir valores de lacolumna 'name' en una cadena separada por comas.
cast_df = cast_df.groupby(cast_df.index)['name'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
#renombrar columna 'name' por 'actors'
cast_df = cast_df.rename(columns = {'name' : 'actors'})
cast_df = cast_df[['actors']]

Se realiza un nuevo join con el credits_data para agregar a cast_df el id de la movie.

In [153]:
cast_df = credits_data[['id']].join(cast_df, how='left')
cast_df.head()

,id,actors
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [154]:
type(cast_df['id'][2])

numpy.int64

### Crew 👨‍🔧
En la exploración inicial y en el diccionario de los datos nos muestran que la columna 'crew' viene como una lista de diccionarios ya que una pelicula puede ser clasificada dentro de varios generos, para proceder a desanidarlo lo primero que se realiza es guardar dentro de una variable independiente las columnas generadas al aplicar el metodo explode el  cual nos ayuda a sacar cada elemento de la lista de diccionarios y los almacena en un dataframe, despues aplicamos a ese diccionario el metodo Series.

In [155]:
#Con el metodo explode() se transforma el dato anidado
crew_df = credits_data['crew'].explode().apply(pd.Series)
crew_df.head()

,credit_id,department,gender,id,job,name,profile_path,0
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,NaN
0,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,NaN
0,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,NaN
0,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,NaN
0,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,NaN


De todas estos datos solo vamos a utilizar a los directores por lo tanto debemos filtrarlos por 'job', luego para los casos en los que se tengan varios directores se van a dejar en un str separados por coma.

In [156]:
#Filtramos el dataframe para solo tener los directores conseravando el indice para dspues unirlos
crew_df = crew_df[crew_df['job'] == 'Director']
#Agrupamos y unimos
crew_df = crew_df.groupby(crew_df.index)['name'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()
#Obtenemos solo la columna 'name'
crew_df = crew_df[['name']]
#renombramos 'name' por 'directors'
crew_df = crew_df.rename(columns = {'name' : 'directors'})

In [157]:
crew_df.head()

,directors
0,John Lasseter
1,Joe Johnston
2,Howard Deutch
3,Forest Whitaker
4,Charles Shyer


Ahora concatenamos los dos dataframes

In [158]:
# Concatenar los DataFrames
credits_data = pd.concat([crew_df, cast_df], axis=1)
credits_data.head()

,directors,id,actors
0,John Lasseter,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,Joe Johnston,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,Howard Deutch,15602.0,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,Forest Whitaker,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,Charles Shyer,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


In [159]:
credits_data.dropna(inplace=True)

In [160]:
credits_data.head()

,directors,id,actors
0,John Lasseter,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,Joe Johnston,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,Howard Deutch,15602.0,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,Forest Whitaker,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,Charles Shyer,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


Vemos que id no está como int asi que lo convertimos.

In [161]:
# Convertir finalmente a entero
credits_data['id'] = credits_data['id'].astype(int)

In [162]:
credits_data.head()

,directors,id,actors
0,John Lasseter,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,Joe Johnston,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,Howard Deutch,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,Forest Whitaker,31357,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,Charles Shyer,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


Traemos el dataset de movies.

In [163]:
# Usando la función read_csv de Pandas leemos y almacenamos movies_dataset como un dataframe con el nombre movies_data.
movies_data = pd.read_csv('../datasets/movies_dataset_final.csv')

Unimos los datasets.

In [164]:
# Unir los DataFrames en base a la columna 'id'
complete_data = pd.merge(movies_data, credits_data, on='id', how='inner')

In [165]:
complete_data.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_count,collection_name,genres,production_companies,production_countries,spoken_languages,return,release_year,directors,actors
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,5415.0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English,12.451801,1995,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,2413.0,NaN,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français",4.043035,1995,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,92.0,Grumpy Old Men Collection,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English,0.000000,1995,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,34.0,NaN,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English,5.090760,1995,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,173.0,Father of the Bride Collection,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English,0.000000,1995,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb..."


Una de las columnas que se van a usar para el consumo de la API será popularity y ahora me fijo que hay que formatearlo.

In [166]:
#Conviertir la variable de tipo str a tipo numérico
complete_data['popularity'] = pd.to_numeric(complete_data['popularity'], errors='coerce')

### Dataset para API 📊
Hay que dejar solo las columnas necesarias para el consumo de la API, se seleccionan estas columnas porque son las relevantes para las funciones que nos requieren.

In [167]:
API_data = complete_data[['budget', 'revenue', 'popularity' ,'release_date', 'title', 'vote_average', 'vote_count', 'actors',
              'release_year', 'directors', 'return' ]]

Ahora procedemos a exportar el dataset en formato parquet.

In [168]:
#Exportar el API_data para consumo de la API en formato.parquet
API_data.to_parquet('../datasets/API_data.parquet', index = False)
#Exportar el complete_data para modelo ML y EDA
complete_data.to_parquet('../datasets/complete_data.parquet')